In [1]:
import zarr
import dask_zarr as dz

### Modify the url to point to the correct location of the zarr file

In [2]:
z_url = r"/mnt/KOMP_C8565_1.zarr"

In [3]:
z = zarr.open(z_url, mode="r") # open the zarr created above in jupyter kernel

### Set up the ImJoy viewer extension

In [4]:
from imjoy import api
import zarr


def encode_zarr_store(zobj):
    path_prefix = f"{zobj.path}/" if zobj.path else ""

    def getItem(key, options = None):
        return zobj.store[path_prefix + key]

    def setItem(key, value):
        zobj.store[path_prefix + key] = value

    def containsItem(key, options = None):
        if path_prefix + key in zobj.store:
            return True

    return {
        "_rintf": True,
        "_rtype": "zarr-array" if isinstance(zobj, zarr.Array) else "zarr-group",
        "getItem": getItem,
        "setItem": setItem,
        "containsItem": containsItem,
    }


api.registerCodec(
    {"name": "zarr-array", "type": zarr.Array, "encoder": encode_zarr_store}
)
api.registerCodec(
    {"name": "zarr-group", "type": zarr.Group, "encoder": encode_zarr_store}
)


class Plugin:
    def __init__(self, images, view_state=None):
        if not isinstance(images, list):
            images = [images]
        self.images = images
        self.view_state = view_state

    async def setup(self):
        pass

    async def run(self, ctx):
        viewer = await api.createWindow(
            type="vizarr", src="https://hms-dbmi.github.io/vizarr"
        )
        if self.view_state:
            await viewer.set_view_state(self.view_state)
        for img in self.images:
            await viewer.add_image(img)


def run_vizarr(images, view_state=None):
    api.export(Plugin(images, view_state))

### Access to the group '0' of the zarr file an visualize it

In [5]:
# Create Zarr 
img = { "source": z['0'], "name": "KOMP_test_1" }

# Run vizarr
run_vizarr(img)

<IPython.core.display.Javascript object>